In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, torch
import sys
from dataclasses import dataclass
from pathlib import Path
from typing import Optional, Union, List
import types
import draccus
import numpy as np
import tqdm
from libero.libero import benchmark
import cv2
import wandb
from experiments.robot.libero.libero_utils import get_libero_env

# Append current directory so that interpreter can find experiments.robot
sys.path.append("../..")


2025-07-22 15:26:19.978174: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-22 15:26:19.978208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-22 15:26:19.980219: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-22 15:26:19.989855: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-22 15:26:20.876938: W tensorflow/comp

In [3]:
@dataclass
class GenerateConfig:
    """Configuration for OpenVLA inference on LIBERO tasks."""
    # fmt: off

    #################################################################################################################
    # Model-specific parameters
    #################################################################################################################
    model_family: str = "openvla"                    # Model family
    pretrained_checkpoint: Union[str, Path] = "/mnt/sda/home/zijianwang/openvla/FT_res/openvla-7b-finetuned-libero-10+libero_10_no_noops+b4+lr-0.0005+lora-r48+dropout-0.0--image_aug--2025-07-18_19-26-25"     # Pretrained checkpoint path
    load_in_8bit: bool = False                       # (For OpenVLA only) Load with 8-bit quantization
    load_in_4bit: bool = False                       # (For OpenVLA only) Load with 4-bit quantization

    center_crop: bool = True                         # Center crop? (if trained w/ random crop image aug)

    #################################################################################################################
    # LIBERO environment-specific parameters
    #################################################################################################################
    task_suite_name: str = "libero_10"             # Task suite. Options: libero_spatial, libero_object, libero_goal, libero_10, libero_90
    num_steps_wait: int = 10                         # Number of steps to wait for objects to stabilize in sim
    num_trials_per_task: int = 50                    # Number of rollouts per task

    #################################################################################################################
    # Utils
    #################################################################################################################
    run_id_note: Optional[str] = None                # Extra note to add in run ID for logging
    local_log_dir: str = "./experiments/logs"        # Local directory for eval logs

    use_wandb: bool = False                          # Whether to also log results in Weights & Biases
    wandb_project: str = "YOUR_WANDB_PROJECT"        # Name of W&B project to log to (use default!)
    wandb_entity: str = "YOUR_WANDB_ENTITY"          # Name of entity to log under

    seed: int = 7                                    # Random Seed (for reproducibility)
    device: str = "cuda:1"                           # Device to use for inference

In [4]:
# Initialize configuration
cfg = GenerateConfig()

In [4]:
import json 
id2caption = json.load(open("/mnt/sda/home/zijianwang/openvla/vla-scripts/DPO/data/id2caption.json"))  

In [5]:
# /mnt/sda/home/zijianwang/openvla/rollouts/2025_07_18
import os
import re

# Path to rollouts directory
rollout_dir = "/mnt/sda/home/zijianwang/openvla/rollouts/2025_07_18"

# Regular expression pattern to extract information from filenames
pattern = r"episode=(\d+)--success=(True|False)--task=(.+)\.mp4$"

results = []

# Walk through directory
for filename in os.listdir(rollout_dir):
    match = re.search(pattern, filename)
    if match:
        episode_num = int(match.group(1))
        success = match.group(2) == "True"
        task_desc = match.group(3)
        
        # Find corresponding task suite and task id
        found = False
        for suite_name, tasks in id2caption.items():
            for task_id, caption in tasks.items():
                if task_desc in caption.replace(" ", "_"):
                    found = True
                    # Adjust episode number: convert from global (0-499) to per-task (0-49)
                    # Each task has 50 episodes, so episode_within_task = episode_num % 50
                    episode_within_task = (episode_num-1) % 50
                    results.append({
                        "filename": filename,
                        "episode": episode_within_task,
                        "success": success,
                        "task_suite": suite_name,
                        "task_id": task_id,
                        "task_description": caption
                    })
                    break
            if found:
                break
# Filter successful trajectories
successful_results = [r for r in results if r['success']]
# Save successful results to JSON file

with open('successful_trajectories.json', 'w') as f:
    json.dump(successful_results, f, indent=4)



In [2]:
# Create mapping from 1-200 to 0-49 (repeating pattern)
# 1-50 -> 0-49, 51-100 -> 0-49, 101-150 -> 0-49, 151-200 -> 0-49
episode_mapping = {}
for i in range(1, 201):
    episode_mapping[i] = (i - 1) % 50

print("Episode mapping created:")
print(f"1 -> {episode_mapping[1]}")
print(f"50 -> {episode_mapping[50]}")
print(f"51 -> {episode_mapping[51]}")
print(f"100 -> {episode_mapping[100]}")
print(f"101 -> {episode_mapping[101]}")
print(f"150 -> {episode_mapping[150]}")
print(f"151 -> {episode_mapping[151]}")
print(f"200 -> {episode_mapping[200]}")


Episode mapping created:
1 -> 0
50 -> 49
51 -> 0
100 -> 49
101 -> 0
150 -> 49
151 -> 0
200 -> 49
